In [ ]:
import pandas as pd
import numpy as np
import re
import datetime

pd.set_option('display.max_colwidth', -1)

In [ ]:
inspections = pd.read_csv("prohlidky.csv")
inspections = inspections.drop(['Unnamed: 0'], axis=1)
inspections.DatKont = pd.to_datetime(inspections.DatKont)
inspections.DatPrvReg = pd.to_datetime(inspections.DatPrvReg)
display(inspections.head())
display(inspections.info())

In [ ]:
stations = pd.read_csv("stk_form_pg.csv")
# stations = stations.drop(['Unnamed: 0'], axis=1)
display(stations.head())
display(stations.info())

In [ ]:
inspections_split = np.array_split(inspections, 1000)
out_of_working_hours = pd.DataFrame(data=None, columns=inspections.columns)
out_of_working_hours['pracovnidoba'] = np.nan
out_of_working_hours['duvod'] = np.nan

In [ ]:
stations.loc[stations['pracovnidoba'] == "Nákladní auta", 'pracovnidoba'] = "0"
stations.loc[stations['pracovnidoba'] == "Pouze Nákladní a autobusy", 'pracovnidoba'] = "0"
stations['stk_id'] = stations['stk_id'].apply(lambda x: x if 4 == len(str(x)) else x * 10)
daysofweek = [1, 2, 3, 4, 5, 6, 0]
daysofweekstr = {
  1: "Po: ",
  2: "Út: ",
  3: "St: ",
  4: "Čt: ",
  5: "Pá: ",
  6: "So: ",
  0: "Ne: "
}

a = 0

for part in inspections_split:
    for index, row in part.iterrows():
        for dayofweek in daysofweek:
            if row.DatKont.weekday() == dayofweek:
                try:
                    if len(stations.where(stations['stk_id'] == row['STK']).dropna(how='all').pracovnidoba.all()) > 10:
                        pracovnidoba = stations.where(stations['stk_id'] == row['STK']).dropna(how='all').pracovnidoba.to_string()
                        index = pracovnidoba.find(daysofweekstr[dayofweek])
                        if index != -1:
                            pattern = re.compile('(.)*{0}(\d):(\d\d)+-(\d\d):(\d\d).*'.format(daysofweekstr[dayofweek]))
                            match = pattern.match(str(pracovnidoba))
                            if match:
                                insp_time = row['DatKont'].hour * 60 + row['DatKont'].minute
                                before = int(match.group(2)) * 60 + int(match.group(3))
                                after = int(match.group(4)) * 60 + int(match.group(5))
                                if insp_time < before or insp_time > after:
                                    row['pracovnidoba'] = pracovnidoba
                                    row['duvod'] = "OUT OF TIME RANGE"
                                    out_of_working_hours = out_of_working_hours.append(row)
                            else:
                                row['pracovnidoba'] = pracovnidoba
                                row['duvod'] = "NO MATCH"
                                out_of_working_hours = out_of_working_hours.append(row)
                        else:
                            row['pracovnidoba'] = pracovnidoba
                            row['duvod'] = "OUT OF WEEKDAY: {0}".format(row.DatKont.weekday())
                            out_of_working_hours = out_of_working_hours.append(row)
                except:
                    print(row)
    a += 1
    print("part: {0}".format(a))

In [ ]:
#stations.sample().pracovnidoba.astype(str).str.find("Po:")
#type(inspections.sample().DatKont.dt)
#stations.where(stations['stk_id'] == 3122).dropna(how='all').loc
#stations.where(stations['stk_id'] == row['STK']).dropna(how='all').pracovnidoba
#type(stations.where(stations['stk_id'] == 3120).dropna(how='all').pracovnidoba.to_string())
#inspections.sample().DatKont.dt.minute
#len(stations.where(stations['stk_id'] == 3120).dropna(how='all').pracovnidoba.all()) > 5
out_of_working_hours

In [ ]:
# stations['Po'] = np.nan
# stations['Ut'] = np.nan
# stations['St'] = np.nan
# stations['Ct'] = np.nan
# stations['Pa'] = np.nan
# stations['So'] = np.nan
# stations['Ne'] = np.nan

# stations['pracovniDoba'] = stations['pracovniDoba'].astype(str).str.replace('Provozní doba','')
# stations['pracovniDoba'] = stations['pracovniDoba'].astype(str).str.replace('Na objednávku','')
# stations['pracovniDoba'] = stations['pracovniDoba'].astype(str).str.replace('Pracovní doba:','')
# stations['pracovniDoba'] = stations['pracovniDoba'].astype(str).str.replace('Po předchozí tel. objednávce','')
# stations['pracovniDoba'] = stations['pracovniDoba'].astype(str).str.replace('Nákladní auta > Dobronická 635\n','')
# stations['pracovniDoba'] = stations['pracovniDoba'].astype(str).str.replace('\r','')
# stations['pracovniDoba'] = stations['pracovniDoba'].astype(str).str.replace('\t','')
# stations['pracovniDoba'] = stations['pracovniDoba'].astype(str).str.replace('\n','')

# for index, row in stations.iterrows():
#     tmp = row['pracovniDoba'].replace(' ', '')
#     if row['pracovniDoba'] == '-':
#             stations.loc[index, 'Po'] = '-'
#             stations.loc[index, 'Ut'] = '-'
#             stations.loc[index, 'St'] = '-'
#             stations.loc[index, 'Ct'] = '-'
#             stations.loc[index, 'Pa'] = '-'
#             stations.loc[index, 'So'] = '-'
#             stations.loc[index, 'Ne'] = '-'
#     if re.match(r'.*Po-Pá:.*', tmp):
#         tmp = tmp[tmp.index('Po-Pá:') + len('Po-Pá:'):]
#         if re.match(r'\d(:|.)\d\d-\d\d(:|.)\d\d.*', tmp):
#             stations.loc[index, 'Po'] = tmp[:10]
#             stations.loc[index, 'Ut'] = tmp[:10]
#             stations.loc[index, 'St'] = tmp[:10]
#             stations.loc[index, 'Ct'] = tmp[:10]
#             stations.loc[index, 'Pa'] = tmp[:10]
            
#             if re.match(r'.*So:.*', tmp[10:]):
#                 tmp = tmp[tmp.index('So:') + len('So:'):]
#                 stations.loc[index, 'So'] = tmp[:10]
            
#             if re.match(r'.*Ne:.*', tmp[10:]):
#                 tmp = tmp[tmp.index('Ne:') + len('Ne:'):]
#                 stations.loc[index, 'Ne'] = tmp[:10]
                
#         if re.match(r'\d-\d\d.*', tmp):
#             stations.loc[index, 'Po'] = tmp[:4]
#             stations.loc[index, 'Ut'] = tmp[:4]
#             stations.loc[index, 'St'] = tmp[:4]
#             stations.loc[index, 'Ct'] = tmp[:4]
#             stations.loc[index, 'Pa'] = tmp[:4]
            
#             if re.match(r'.*So:.*', tmp[4:]):
#                 tmp = tmp[tmp.index('So:') + len('So:'):]
#                 stations.loc[index, 'So'] = tmp[:4]
                
#             if re.match(r'.*Ne:.*', tmp[4:]):
#                 tmp = tmp[tmp.index('Ne:') + len('Ne:'):]
#                 stations.loc[index, 'Ne'] = tmp[:4]
                
#     tmp = row['pracovniDoba'].replace(' ', '')      
#     if re.match(r'.*Po-Čt:.*', tmp):
#         tmp = tmp[tmp.index('Po-Čt:') + len('Po-Čt:'):]
#         if re.match(r'\d(:|.)\d\d-\d\d(:|.)\d\d.*', tmp):
#             stations.loc[index, 'Po'] = tmp[:10]
#             stations.loc[index, 'Ut'] = tmp[:10]
#             stations.loc[index, 'St'] = tmp[:10]
#             stations.loc[index, 'Ct'] = tmp[:10]
            
#             if re.match(r'.*Pá:.*', tmp[10:]):
#                 tmp = tmp[tmp.index('Pá:') + len('Pá:'):]
#                 stations.loc[index, 'Pa'] = tmp[:10]
            
#             if re.match(r'.*So:.*', tmp[10:]):
#                 tmp = tmp[tmp.index('So:') + len('So:'):]
#                 stations.loc[index, 'So'] = tmp[:10]
                
#             if re.match(r'.*Ne:.*', tmp[10:]):
#                 tmp = tmp[tmp.index('Ne:') + len('Ne:'):]
#                 stations.loc[index, 'Ne'] = tmp[:10]
            
#         if re.match(r'\d-\d\d.*', tmp):
#             stations.loc[index, 'Po'] = tmp[:4]
#             stations.loc[index, 'Ut'] = tmp[:4]
#             stations.loc[index, 'St'] = tmp[:4]
#             stations.loc[index, 'Ct'] = tmp[:4]
            
#             if re.match(r'.*Pá:.*', tmp[4:]):
#                 tmp = tmp[tmp.index('Pá:') + len('Pá:'):]
#                 stations.loc[index, 'Pa'] = tmp[:4]
            
#             if re.match(r'.*So:.*', tmp[4:]):
#                 tmp = tmp[tmp.index('So:') + len('So:'):]
#                 stations.loc[index, 'So'] = tmp[:4]
                
#             if re.match(r'.*Ne:.*', tmp[4:]):
#                 tmp = tmp[tmp.index('Ne:') + len('Ne:'):]
#                 stations.loc[index, 'Ne'] = tmp[:4]
                
#     tmp = row['pracovniDoba'].replace(' ', '')               
#     if re.match(r'.*Po,St:.*', tmp):
#         tmp = tmp[tmp.index('Po,St:') + len('Po,St:'):]
#         if re.match(r'\d(:|.)\d\d-\d\d(:|.)\d\d.*', tmp):
#             stations.loc[index, 'Po'] = tmp[:10]
#             stations.loc[index, 'St'] = tmp[:10]
            
#             if re.match(r'.*Pá:.*', tmp[10:]):
#                 tmp = tmp[tmp.index('Pá:') + len('Pá:'):]
#                 stations.loc[index, 'Pa'] = tmp[:10]
            
#             if re.match(r'.*So:.*', tmp[10:]):
#                 tmp = tmp[tmp.index('So:') + len('So:'):]
#                 stations.loc[index, 'So'] = tmp[:10]
                
#             if re.match(r'.*Ne:.*', tmp[10:]):
#                 tmp = tmp[tmp.index('Ne:') + len('Ne:'):]
#                 stations.loc[index, 'Ne'] = tmp[:10]
            
#         if re.match(r'\d-\d\d.*', tmp):
#             stations.loc[index, 'Po'] = tmp[:4]
#             stations.loc[index, 'St'] = tmp[:4]
            
#             if re.match(r'.*Pá:.*', tmp[4:]):
#                 tmp = tmp[tmp.index('Pá:') + len('Pá:'):]
#                 stations.loc[index, 'Pa'] = tmp[:4]
            
#             if re.match(r'.*So:.*', tmp[4:]):
#                 tmp = tmp[tmp.index('So:') + len('So:'):]
#                 stations.loc[index, 'So'] = tmp[:4]
                
#             if re.match(r'.*Ne:.*', tmp[4:]):
#                 tmp = tmp[tmp.index('Ne:') + len('Ne:'):]
#                 stations.loc[index, 'Ne'] = tmp[:4]
                
#     tmp = row['pracovniDoba'].replace(' ', '')                
#     if re.match(r'.*Út,Čt:.*', tmp):
#         tmp = tmp[tmp.index('Út,Čt:') + len('Út,Čt:'):]
#         if re.match(r'\d(:|.)\d\d-\d\d(:|.)\d\d.*', tmp):
#             stations.loc[index, 'Ut'] = tmp[:10]
#             stations.loc[index, 'Ct'] = tmp[:10]
            
#             if re.match(r'.*Pá:.*', tmp[10:]):
#                 tmp = tmp[tmp.index('Pá:') + len('Pá:'):]
#                 stations.loc[index, 'Pa'] = tmp[:10]
            
#             if re.match(r'.*So:.*', tmp[10:]):
#                 tmp = tmp[tmp.index('So:') + len('So:'):]
#                 stations.loc[index, 'So'] = tmp[:10]
                
#             if re.match(r'.*Ne:.*', tmp[10:]):
#                 tmp = tmp[tmp.index('Ne:') + len('Ne:'):]
#                 stations.loc[index, 'Ne'] = tmp[:10]
            
#         if re.match(r'\d-\d\d.*', tmp):
#             stations.loc[index, 'Ut'] = tmp[:4]
#             stations.loc[index, 'Ct'] = tmp[:4]
            
#             if re.match(r'.*Pá:.*', tmp[4:]):
#                 tmp = tmp[tmp.index('Pá:') + len('Pá:'):]
#                 stations.loc[index, 'Pa'] = tmp[:4]
            
#             if re.match(r'.*So:.*', tmp[4:]):
#                 tmp = tmp[tmp.index('So:') + len('So:'):]
#                 stations.loc[index, 'So'] = tmp[:4]
                
#             if re.match(r'.*Ne:.*', tmp[4:]):
#                 tmp = tmp[tmp.index('Ne:') + len('Ne:'):]
#                 stations.loc[index, 'Ne'] = tmp[:4]
                
#     tmp = row['pracovniDoba'].replace(' ', '')                
#     if re.match(r'\d(:|.)\d\d-\d\d(:|.)\d\d.*', tmp):
#         if re.match(r'.*Po:.*', tmp[10:]):
#             tmp = tmp[tmp.index('Po:') + len('Po:'):]
#             stations.loc[index, 'Po'] = tmp[:10]

#         if re.match(r'.*Út:.*', tmp[10:]):
#             tmp = tmp[tmp.index('Út:') + len('Út:'):]
#             stations.loc[index, 'Út'] = tmp[:10]
            
#         if re.match(r'.*St:.*', tmp[10:]):
#             tmp = tmp[tmp.index('St:') + len('St:'):]
#             stations.loc[index, 'St'] = tmp[:10]
        
#         if re.match(r'.*Čt:.*', tmp[10:]):
#             tmp = tmp[tmp.index('Čt:') + len('Čt:'):]
#             stations.loc[index, 'Čt'] = tmp[:10]
            
#         if re.match(r'.*Pá:.*', tmp[10:]):
#             tmp = tmp[tmp.index('Pá:') + len('Pá:'):]
#             stations.loc[index, 'Pa'] = tmp[:10]

#         if re.match(r'.*So:.*', tmp[10:]):
#             tmp = tmp[tmp.index('So:') + len('So:'):]
#             stations.loc[index, 'So'] = tmp[:10]

#         if re.match(r'.*Ne:.*', tmp[10:]):
#             tmp = tmp[tmp.index('Ne:') + len('Ne:'):]
#             stations.loc[index, 'Ne'] = tmp[:10]
            
#     tmp = row['pracovniDoba'].replace(' ', '')
#     if re.match(r'\d-\d\d.*', tmp):
#         if re.match(r'.*Po:.*', tmp[4:]):
#             tmp = tmp[tmp.index('Po:') + len('Po:'):]
#             stations.loc[index, 'Po'] = tmp[:4]

#         if re.match(r'.*Út:.*', tmp[4:]):
#             tmp = tmp[tmp.index('Út:') + len('Út:'):]
#             stations.loc[index, 'Ut'] = tmp[:4]
            
#         if re.match(r'.*St:.*', tmp[4:]):
#             tmp = tmp[tmp.index('St:') + len('St:'):]
#             stations.loc[index, 'St'] = tmp[:4]
        
#         if re.match(r'.*Čt:.*', tmp[4:]):
#             tmp = tmp[tmp.index('Čt:') + len('Čt:'):]
#             stations.loc[index, 'Ct'] = tmp[:4]
#         if re.match(r'.*Pá:.*', tmp[4:]):
#             tmp = tmp[tmp.index('Pá:') + len('Pá:'):]
#             stations.loc[index, 'Pa'] = tmp[:4]

#         if re.match(r'.*So:.*', tmp[4:]):
#             tmp = tmp[tmp.index('So:') + len('So:'):]
#             stations.loc[index, 'So'] = tmp[:4]

#         if re.match(r'.*Ne:.*', tmp[4:]):
#             tmp = tmp[tmp.index('Ne:') + len('Ne:'):]
#             stations.loc[index, 'Ne'] = tmp[:4]
            

# stations['Po'] = stations['Po'].astype(str).str.replace(':00','')
# stations['Ut'] = stations['Ut'].astype(str).str.replace(':00','')
# stations['St'] = stations['St'].astype(str).str.replace(':00','')
# stations['Ct'] = stations['Ct'].astype(str).str.replace(':00','')
# stations['Pa'] = stations['Pa'].astype(str).str.replace(':00','')
# stations['So'] = stations['So'].astype(str).str.replace(':00','')
# stations['Ne'] = stations['Ne'].astype(str).str.replace(':00','')